In [3]:
from detecterreur.form.form_diacritic import FormDiacritic

In [4]:
import pandas as pd
import numpy as np
from sentence_transformers import SentenceTransformer
from transformers import AutoModelForCausalLM, AutoTokenizer

/home/monicapsq/Desktop/detecterreur/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Using FDIA as an example

In [5]:
dia = FormDiacritic()
sentence = "Je suis francais"

In [6]:
error = dia.get_error(sentence)

In [7]:
print(error)

(True, 'FDIA')


In [8]:
error_name = error[1]
error[1]

'FDIA'

# Embedding the rules

In [10]:
df = pd.read_csv('rules/rules_test2.csv', sep=';')
display(df)

,"rules,"
0,"FORME FAGL Lendemain instead of l’en demain, ,"
1,FORME FMAJ Si le titre commence par un article...
2,FORME FMAJ Si le titre commence par l’article ...
3,FORME FMAJ Quand le titre ne commence pas par ...
4,FORME FMAJ Quand le titre est composé d’un ou ...
...,...
58,"SYNTAXE SORD,"
59,"SYNTAXE SINS,"
60,"SYNTAXE SRED,"
61,"SYNTAXE SMIS,"


In [12]:
df.shape

(63, 1)

In [11]:
# Load embedding model
embedding_model = SentenceTransformer('dangvantuan/sentence-camembert-base')

In [12]:
# Generate embeddings for the 'rules' column, add them as a second column
df['embeddings'] = df['rules,'].apply(lambda x: embedding_model.encode(x))

In [13]:
df

,"rules,",embeddings
0,"FORME FAGL Lendemain instead of l’en demain, ,","[0.14340821, 0.036741666, -0.019806538, 0.0427..."
1,FORME FMAJ Si le titre commence par un article...,"[-0.010824608, 0.21325883, -0.1582906, 0.08447..."
2,FORME FMAJ Si le titre commence par l’article ...,"[0.008349443, 0.13198903, -0.13986641, 0.13734..."
3,FORME FMAJ Quand le titre ne commence pas par ...,"[-0.018213877, 0.14898089, -0.123592354, 0.050..."
4,FORME FMAJ Quand le titre est composé d’un ou ...,"[-0.00965369, 0.013733, -0.13376188, 0.1273915..."
...,...,...
58,"SYNTAXE SORD,","[-0.015337427, -0.12299016, 0.12389943, 0.1609..."
59,"SYNTAXE SINS,","[-0.12720205, 0.4196547, -0.015587378, 0.11928..."
60,"SYNTAXE SRED,","[0.02234101, 0.10750864, 0.0470725, 0.10789981..."
61,"SYNTAXE SMIS,","[-0.024400063, 0.15761055, -0.042611405, 0.151..."


# Embedding the prompt

In [14]:
prompt1 = f"""Vous produisez le retour d'un outil de détection et de correction d'erreures dans des productions écrites de Français comme langue étrangèere (FLE). L'étudiant.e vient de faire l'erreur: {error_name}. Fournissez une correction pour l'erreur trouvé par l'outil. Utilisez un langage simple et adapté à des étudiant.e.s de français."""
prompt1

"Vous produisez le retour d'un outil de détection et de correction d'erreures dans des productions écrites de Français comme langue étrangèere (FLE). L'étudiant.e vient de faire l'erreur: FDIA. Fournissez une correction pour l'erreur trouvé par l'outil. Utilisez un langage simple et adapté à des étudiant.e.s de français."

In [15]:
prompt_embedding = embedding_model.encode(prompt1)

In [16]:
prompt_embedding[0:10]

array([ 0.01169323, -0.19686155, -0.06162453,  0.05953577,  0.09079259,
       -0.0077934 ,  0.1246677 , -0.0625868 , -0.10314687,  0.05594119],
      dtype=float32)

# Getting the distances

In [17]:
file_embeddings = df['embeddings']

In [18]:
def dist(file_embeddings):
  return np.dot(file_embeddings, prompt_embedding)

In [19]:
df['distance'] = df['embeddings'].apply(dist)

In [20]:
df

,"rules,",embeddings,distance
0,"FORME FAGL Lendemain instead of l’en demain, ,","[0.14340821, 0.036741666, -0.019806538, 0.0427...",1.540608
1,FORME FMAJ Si le titre commence par un article...,"[-0.010824608, 0.21325883, -0.1582906, 0.08447...",1.185553
2,FORME FMAJ Si le titre commence par l’article ...,"[0.008349443, 0.13198903, -0.13986641, 0.13734...",1.282351
3,FORME FMAJ Quand le titre ne commence pas par ...,"[-0.018213877, 0.14898089, -0.123592354, 0.050...",0.890028
4,FORME FMAJ Quand le titre est composé d’un ou ...,"[-0.00965369, 0.013733, -0.13376188, 0.1273915...",0.669711
...,...,...,...
58,"SYNTAXE SORD,","[-0.015337427, -0.12299016, 0.12389943, 0.1609...",0.954958
59,"SYNTAXE SINS,","[-0.12720205, 0.4196547, -0.015587378, 0.11928...",0.500031
60,"SYNTAXE SRED,","[0.02234101, 0.10750864, 0.0470725, 0.10789981...",1.304347
61,"SYNTAXE SMIS,","[-0.024400063, 0.15761055, -0.042611405, 0.151...",0.272191


In [21]:
# sort ascending
df.sort_values('distance', ascending=False, inplace=True)
df

,"rules,",embeddings,distance
31,FORME FDIA les rectifications de l'orthographe...,"[-0.020076357, 0.087852105, -0.13033451, 0.115...",2.323813
7,FORME FDIA les mots en -ère et -ès prennent un...,"[0.07750241, 0.07635148, -0.19300944, 0.171116...",2.114066
19,FORME FDIA on ne met jamais d’accent grave sur...,"[0.023513332, -0.020197302, -0.19267227, 0.136...",1.989851
6,FORME FDIA On utilise un accent aigu lorsque l...,"[0.026381858, 0.02602192, -0.21559115, 0.12802...",1.986505
14,FORME FDIA on ne met jamais d'accent aigu sur ...,"[-0.014018504, -0.1114261, -0.19020045, 0.1292...",1.957395
...,...,...,...
36,"ORTOGRAPHE OSUB,","[-0.19843645, 0.014199812, 0.011061102, 0.0266...",0.574719
62,"PONCTUATION PONC ,","[-0.06955011, 0.049458392, 0.018116213, -0.027...",0.519269
59,"SYNTAXE SINS,","[-0.12720205, 0.4196547, -0.015587378, 0.11928...",0.500031
61,"SYNTAXE SMIS,","[-0.024400063, 0.15761055, -0.042611405, 0.151...",0.272191


# Creating the context

In [22]:
top_rows = df.head(10)
context_list = top_rows['rules,'].tolist()
context = "\n".join(context_list)

print(context)

FORME FDIA les rectifications de l'orthographe de 1990 préconisent la suppression de l'accent circonflexe sur le u et le i.
Exemples : la chaine – la voute – paraitre – il parait,
FORME FDIA les mots en -ère et -ès prennent un accent grave : Une ère, un ers (légume lentille)
,
FORME FDIA on ne met jamais d’accent grave sur les voyelles « e » précédant un « x ».
Exemples : un accent circonflexe, le sexe.
,
FORME FDIA On utilise un accent aigu lorsque la voyelle « e » est la première lettre du mot
Exemples : étable, élection, étendre…,
FORME FDIA on ne met jamais d'accent aigu sur les voyelles « e » précédant un « x ».
Exemples : un accent circonflexe, le sexe.,
FORME FDIA on met un accent aigu sur un « e » précédant une syllabe sans "e" muet
Exemples : immédiat – compléter – génération – généralement,
FORME FDIA on utilise un accent circonflexe sur les « o » des mots possessifs
Exemples : le nôtre, le vôtre, les nôtres, les vôtres.,
FORME FDIA La cédille (¸) ne s'emploie qu'avec la lett

# Generating the output

Salomon said it would be even better to put both `{error_macrocategory}` and `{error_name}` but since DetectErreur doesn't do it yet I didn't implement it this way

In [25]:
# prompt2 = f"""Vous produisez le retour d'un outil de détection et de correction d'erreures dans des productions écrites de Français comme langue étrangèere (FLE). L'étudiant.e vient de faire l'erreur: {error_name}. Fournissez une correction pour l'erreur trouvé par l'outil aidé par le contexte: {context}. Utilisez un langage simple et adapté à des étudiants de français."""

In [24]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen2.5-1.5B")

In [ ]:
messages = [
    {"role": "user", "content": prompt2},
]
inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

In [32]:
outputs = model.generate(
    **inputs, 
    max_new_tokens=200, 
    do_sample=True, 
    temperature=0.7, 
    repetition_penalty=1.2 
)

print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


桤<|endoftext|>


In [35]:
# 1. Define the structured prompt
prompt2 = f"""### Rôle
Tu es un professeur de français (FLE) pédagogue.

### Analyse
Phrase de l'élève : "{sentence}"
Erreur : {error_name}

### Contexte
{context}

### Consignes
Utilise le contexte pour expliquer l'erreur, mais reformule-le avec tes propres mots.
1. Donne la phrase corrigée.
2. Explique la règle simplement.
3. Sois encourageant.
4. Maximum 200 mots.

### Réponse :
"""

# 2. Prepare the inputs (This must be re-run to use the new prompt2)
messages = [{"role": "user", "content": prompt2}]
inputs = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    tokenize=True,
    return_dict=True,
    return_tensors="pt",
).to(model.device)

# 3. Generate with repetition_penalty to prevent "桤" loops
outputs = model.generate(
    **inputs, 
    max_new_tokens=200, 
    do_sample=True, 
    temperature=0.7, 
    repetition_penalty=1.2  # This is the key fix for the symbols
)

# 4. Decode and print
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True))


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


桤acette
桤acouleurpapier
